<a href="https://colab.research.google.com/github/andreykoz82/KRLS/blob/master/linear_regression_monthly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# АНАЛИЗ ПРОИЗВОДСТВА РОМАШКИ ЦВЕТКИ ФП

In [2]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
import plotly.graph_objects as go
import plotly.express as px
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/KLS/Chamomile")

Загрузка исходных данных за период с 2012 по 2020 г.

In [5]:
df = pd.read_excel('data_2012_2020_monthly.xlsx')
data = df.loc[df['Расход сырья'] > 0]
data['Цена сырья'] = (data['Стоимость сырья'] / data['Расход сырья'])
data['Себестоимость НФП'] = (data['Стоиомсть НФП'] / data['Выпуск НФП'])
data['Себестоимость ГП'] = (data['Стоимость ГП'] / data['Выпуск ГП'])
data['Стоимость отходов'] = (data['Отходы'] * data['Цена сырья'])
data['Процент отходов'] = (data['Отходы'] / data['Расход сырья'] * 100)
data['Цена материалы'] = (data['Вспомогательные материалы'] / data['Выпуск ГП'])
data['Сырье на пачку'] = (data['Стоимость сырья'] / data['Выпуск ГП'])
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,Дата,Расход сырья,Стоимость сырья,Отходы,Выпуск НФП,Стоиомсть НФП,Выпуск ГП,Стоимость ГП,Вспомогательные материалы,Цена сырья,Себестоимость НФП,Себестоимость ГП,Стоимость отходов,Процент отходов,Цена материалы,Сырье на пачку
2,2012-03-01,9425.000,1440624.15,660.6,38181.6,7675947.63,1505475,13598004.93,3631289.47,152.851369,201.037872,9.032368,100973.614163,7.009019,2.412056,0.956923
3,2012-04-01,14499.181,2216219.66,1313.2,12940.6,2308665.27,986247,8326135.66,2277574.84,152.851369,178.404809,8.442242,200724.417297,9.057063,2.309335,2.247124
4,2012-05-01,18788.742,2871884.93,1777.1,18927.8,3349803.67,520625,4124465.53,1189371.89,152.851369,176.977973,7.922143,271632.167236,9.458324,2.284508,5.516226
5,2012-06-01,19412.400,2963067.54,2193.7,16817.3,3063068.16,412914,3320219.59,923760.85,152.637878,182.137927,8.040947,334841.712642,11.300509,2.237175,7.175992
6,2012-07-01,20713.100,2818379.93,1854.1,28395.0,4948404.32,751957,6129327.94,1723780.68,136.067509,174.270270,8.151168,252282.769272,8.951340,2.292393,3.748060


In [6]:
def make_regression(feature_1: str, feature_2: str, target: str, degree: int):
    X = data[[feature_1, feature_2]]
    y = data[target]
    
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X, y)
    
    y_hat = model.predict(X)
    r2 = r2_score(y, y_hat).round(2)
    
    x_surf, y_surf = np.meshgrid(np.linspace(data[feature_1].min(), data[feature_1].max(), 100),
                             np.linspace(data[feature_2].min(), data[feature_2].max(), 100))
    X_train = pd.DataFrame({feature_1: x_surf.ravel(), feature_2: y_surf.ravel()})
    y_pred = model.predict(X_train)
    
    fig = go.Figure(data=[go.Scatter3d(x=data[feature_1], y=data[feature_2], z=data[target],
                                   mode='markers',marker=dict(size=3, sizemode='diameter')),
                     go.Surface(x=x_surf,y=y_surf, z=y_pred.reshape(x_surf.shape))])
    fig.update_layout(
        title=f"Зависимость себестоимости ГП от {feature_1} и {feature_2} (качество модели R^2 = {r2})",
        scene = dict(
        xaxis_title=feature_1,
        yaxis_title=feature_2,
        zaxis_title=target),
        margin=dict(l=40, r=20, t=40, b=40))

    fig.update_traces(opacity=0.5, selector=dict(type='surface'))   
    fig.show()

In [7]:
make_regression('Цена сырья', 'Процент отходов', 'Себестоимость ГП', degree=2)

In [8]:
make_regression('Сырье на пачку', 'Цена материалы', 'Себестоимость ГП', degree=3)

In [9]:
@interact(raw_mat_cost = (100.0, 300.0), 
          sup_mat_cost = (0.0, 10.0),
          waste_perc = (0.0, 15.0))

def regression(raw_mat_cost, sup_mat_cost, waste_perc, poly_degree=1):
    X = data[['Цена сырья', 'Цена материалы', 'Процент отходов']]
    y = data['Себестоимость ГП']
    
    poly_features = PolynomialFeatures(poly_degree)
    X = poly_features.fit_transform(X)
    
    model = LinearRegression()
    model.fit(X, y)
    print("Model R^2:", r2_score(y, model.predict(X)).round(3))
    
    X_test = poly_features.fit_transform(np.array([raw_mat_cost, sup_mat_cost, waste_perc]).reshape(-1, 3))
    y_hat = model.predict(X_test)
    print()
    print("Коэффициенты уравнения апроксимирующей функции:", [x.round(3) for x in model.coef_ if x > 0])
    print("Смещение:", model.intercept_.round(3))
    print()
    return print("Себестоимость ГП:", *y_hat.round(2))

interactive(children=(FloatSlider(value=200.0, description='raw_mat_cost', max=300.0, min=100.0), FloatSlider(…